# Hand Gesture Recognition - Model Training and Evaluation
## Building CNN and CNN+LSTM Models

This notebook covers:
- Preprocessing collected data
- Building CNN model for static gesture recognition
- Building CNN+LSTM model for temporal gesture recognition
- Training with proper validation and callbacks
- Evaluating model performance
- Visualizing results with confusion matrices and training history

## 1. Setup and Import Libraries

In [ ]:
import sys
import os
from pathlib import Path
import warnings

sys.path.insert(0, str(Path('../').resolve()))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TF warnings

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

warnings.filterwarnings('ignore')

# Check TensorFlow version
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ GPU Available: {tf.config.list_physical_devices('GPU')}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## 2. Load and Prepare Dataset

In [ ]:
from src.preprocessing import HandPreprocessor, load_dataset, save_dataset

# Preprocess raw data
print("Loading and preprocessing dataset...")
print("-" * 50)

preprocessor = HandPreprocessor(image_size=(224, 224))

try:
    # Try to load preprocessed dataset
    X_train, X_test, y_train, y_test, class_names, class_to_idx = load_dataset('../data/processed/dataset.pkl')
    print("✓ Preprocessed dataset loaded from cache")
except FileNotFoundError:
    # Preprocess from raw data
    print("Preprocessing raw data...")
    X_train, X_test, y_train, y_test, class_names, class_to_idx = \
        preprocessor.load_dataset_from_directory('../data/raw', augment=True, test_split=0.2)
    
    # Save for future use
    save_dataset(X_train, X_test, y_train, y_test, class_names, class_to_idx)

# Split train into train and validation
split_idx = int(len(X_train) * 0.8)
X_val = X_train[split_idx:]
y_val = y_train[split_idx:]
X_train = X_train[:split_idx]
y_train = y_train[:split_idx]

print(f"\nDataset Summary:")
print(f"  Classes: {class_names}")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")
print(f"  Input shape: {X_train[0].shape}")
print(f"  Label range: {y_train.min()} - {y_train.max()}")

## 3. Build CNN Model

In [ ]:
from src.models import create_simple_cnn_model, GestureRecognitionCNN

# Create a simple CNN model
print("Building CNN Model...")
print("-" * 50)

model_cnn = create_simple_cnn_model(num_classes=len(class_names))

# Compile model
model_cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
        keras.metrics.SparseTopKCategoricalAccuracy(k=2, name='top_2_accuracy')
    ]
)

print("✓ Model compiled successfully")
print(f"✓ Total parameters: {model_cnn.count_params():,}")

# Show model architecture
model_cnn.summary()

## 4. Train CNN Model

In [ ]:
# Setup callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        '../models/best_cnn_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=0
    )
]

# Train model
print("Training CNN Model...")
print("-" * 50)

history_cnn = model_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Training completed!")

## 5. Evaluate Model Performance

In [ ]:
# Evaluate on test set
print("Evaluating Model on Test Set...")
print("-" * 50)

test_loss, test_accuracy, test_top2 = model_cnn.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Top-2 Accuracy: {test_top2:.4f}")

# Get predictions
y_pred_probs = model_cnn.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

# Visualize training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy
axes[0].plot(history_cnn.history['accuracy'], label='Training', linewidth=2)
axes[0].plot(history_cnn.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=11, fontweight='bold')
axes[0].set_title('Model Accuracy', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3)

# Loss
axes[1].plot(history_cnn.history['loss'], label='Training', linewidth=2)
axes[1].plot(history_cnn.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Loss', fontsize=11, fontweight='bold')
axes[1].set_title('Model Loss', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=class_names,
           yticklabels=class_names,
           cbar_kws={'label': 'Count'},
           annot_kws={'size': 12, 'weight': 'bold'})
plt.xlabel('Predicted Label', fontsize=11, fontweight='bold')
plt.ylabel('True Label', fontsize=11, fontweight='bold')
plt.title('Confusion Matrix - Test Set', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Prediction Visualization with Confidence Scores

In [ ]:
# Visualize predictions with confidence scores
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
axes = axes.flatten()

# Select 15 random test samples
sample_indices = np.random.choice(len(X_test), 15, replace=False)

for idx, sample_idx in enumerate(sample_indices):
    # Get image and prediction
    image = X_test[sample_idx]
    true_label = y_test[sample_idx]
    pred_label = y_pred[sample_idx]
    confidence = y_pred_probs[sample_idx][pred_label]
    
    # Display image
    axes[idx].imshow(image)
    axes[idx].axis('off')
    
    # Set title with prediction info
    true_name = class_names[true_label]
    pred_name = class_names[pred_label]
    
    color = 'green' if true_label == pred_label else 'red'
    title = f"True: {true_name}\nPred: {pred_name}\nConf: {confidence:.2%}"
    axes[idx].set_title(title, fontsize=9, fontweight='bold', color=color)

plt.suptitle('Sample Predictions with Confidence Scores', fontsize=13, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Per-class accuracy
print("\nPer-Class Accuracy:")
print("-" * 40)
for class_idx, class_name in enumerate(class_names):
    class_mask = y_test == class_idx
    if class_mask.sum() > 0:
        class_accuracy = accuracy_score(y_test[class_mask], y_pred[class_mask])
        print(f"{class_name:15s}: {class_accuracy:.4f} ({int(class_mask.sum())} samples)")

## 7. Save Model

In [ ]:
# Save final model
model_path = Path('../models/cnn_gesture_model.h5')
model_path.parent.mkdir(parents=True, exist_ok=True)
model_cnn.save(str(model_path))

print(f"✓ Model saved to: {model_path}")
print(f"  Model size: {model_path.stat().st_size / 1e6:.2f} MB")

# Save class information
import json
class_info = {
    'class_names': class_names,
    'class_to_idx': class_to_idx,
    'test_accuracy': float(test_accuracy),
    'test_loss': float(test_loss)
}

info_path = Path('../models/cnn_class_info.json')
with open(info_path, 'w') as f:
    json.dump(class_info, f, indent=2)

print(f"✓ Class information saved to: {info_path}")